## Basics of pyspark from DataCamp

This notebook follows 

- https://campus.datacamp.com/courses/introduction-to-pyspark/getting

In [1]:
import pyspark

## Spark context

- **You can connect to a Spark cluster using `SparkContext` class**


- Creating the connection to a cluster is as simple as creating an instance of the SparkContext class. 


- The class constructor takes a few optional arguments that allow you to specify the attributes of the cluster you're connecting to.




In [2]:
# It is possible that the context sc is automatically imported
# once pyspark is imported
sc = SparkContext.getOrCreate()

In [3]:
sc

<SparkContext master=local[2] appName=PySparkShell>

In [4]:
#help(pyspark.SparkContext)

## Spark Configuration

- `sc._conf`: Configuration for a Spark application. Used to set various Spark parameters as key-value pairs. Once a SparkConf object is passed to Spark, it is cloned and **can no longer be modified by the user**.


In [5]:
from pyspark.sql import SparkSession

In [7]:
sc._conf.getAll() 

[('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1527419876564'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.host', '192.168.1.33'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.port', '53835'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.master', 'local[2]')]

## Pyspark Dataframe 

Pyspark has has a native dataframe type

In [46]:
sdf = spark.read.csv("../data_samples/iris.csv", header=True)

In [47]:
type(sdf)

pyspark.sql.dataframe.DataFrame

In [49]:
sdf.columns

['150', '4', 'setosa', 'versicolor', 'virginica']

In [50]:
sdf.head(2)

[Row(150='5.1', 4='3.5', setosa='1.4', versicolor='0.2', virginica='0'),
 Row(150='4.9', 4='3.0', setosa='1.4', versicolor='0.2', virginica='0')]

In [55]:
sdf.count()

150

### Cast Pyspark dataframe to pandas dataframe

In [53]:
df = sdf.toPandas();

In [54]:
df.head()

,150,4,setosa,versicolor,virginica
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


## Creating a SparkSession

- Creating multiple `SparkSessions` and `SparkContexts` can cause issues, so it's best practice to use the `SparkSession.builder.getOrCreate()` method. 


- This returns an existing `SparkSession` if there's already one in the environment, or creates a new one if necessary.


In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.getOrCreate()

In [7]:
spark

## Viewing tables

- Once you've created a **`SparkSession`**, you can start poking around to see what data is in your cluster!


- Your **`SparkSession`** has an attribute called catalog which lists all the data inside the cluster. This attribute has a few methods for extracting different pieces of information.


- One of the most useful is the **`.listTables()`** method, which returns the names of all the tables in your cluster as a list.




In [8]:
print(spark.catalog.listTables())

[]


## Queries 

- One of the advantages of the DataFrame interface is that you can run SQL queries on the tables in your Spark cluster.


- As you saw in the last exercise, one of the tables in your cluster is the `flights` table. This table contains a row for every flight that left Portland International Airport (PDX) or Seattle-Tacoma International Airport (SEA) in 2014 and 2015.

- Running a query on this table is as easy as using the **`.sql()`** method on your `SparkSession` that we called `spark`. This method takes a string containing the query and returns a DataFrame with the results!

- If you look closely, you'll notice that the table `flights` is only mentioned in the query, not as an argument to any of the methods. This is because there isn't a local object in your environment that holds that data, so it wouldn't make sense to pass the table as an argument.

- Remember, we've already created a SparkSession called spark in your workspace.

```python
#Don't change this query
query = "FROM flights SELECT * LIMIT 10"

#Get the first 10 rows of flights
flights10 = spark.sql(query)

#Show the results
flights10.show()
```


In [ ]:
# Don't change this query
query = "FROM flights SELECT * LIMIT 10"

# Get the first 10 rows of flights
#flights10 = spark.sql(query)

# Show the results
#flights10.show()

## `.toPandas()` a Spark Dataframe

- Suppose you've run a query on your huge dataset and aggregated it down to something a little more manageable.


- Sometimes it makes sense to then take that table and work with it locally using a tool like `pandas`. Spark DataFrames make that easy with the **`.toPandas()`** method. Calling this method on a Spark DataFrame returns the corresponding `pandas` DataFrame. It's as simple as that!


## Pandas dataframe to SparkCluster

- The `SparkSession class` has **`.createDataFrame()`** that takes a pandas DataFrame and returns a Spark DataFrame.


- The output of this method is stored locally, not in the `SparkSession` catalog. This means that you can use all the Spark DataFrame methods on it, but you can't access the data in other contexts.


- For example, a SQL query (using the `.sql()` method) that references your DataFrame will throw an error. To access the data in this way, you have to save it as a temporary table.


- You can do this using the **`.createTempView()`** Spark DataFrame method, which takes as its only argument the name of the temporary table you'd like to register. This method registers the DataFrame as a table in the catalog, but as this table is temporary, it can only be accessed from the specific `SparkSession` used to create the Spark DataFrame.


- There is also the method **`.createOrReplaceTempView()`**. This safely creates a new temporary table if nothing was there before, or updates an existing table if one was already defined. You'll use this method to avoid running into problems with duplicate tables.


![SparkDiagram](./images/spark_figure.png)

In [ ]:
import pandas as pd

In [ ]:
pyspark.Da

In [ ]:
# Create pd_temp
pd_temp = pd.DataFrame(np.random.random(10))

# Create spark_temp from pd_temp
spark_temp = spark.createDataFrame(pd_temp)

# Examine the tables in the catalog
print(spark.catalog.listTables())

# Add spark_temp to the catalog
spark_temp.createOrReplaceTempView("temp")

# Examine the tables in the catalog again
print(spark.catalog.listTables())